<a href="https://colab.research.google.com/github/ufrpe-mineracao-textos/projeto-de-mineracao-20192-ner-analise-juridica/blob/master/ner_juridico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import spacy
from spacy import displacy
import os
import json
import logging
import random

In [0]:
os.system("python -m spacy download pt")

0

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Converter para o formato Spacy

In [0]:
def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                #only a single point in text annotation.
                point = annotation['points'][0]
                labels = annotation['label']
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        return training_data
    except Exception as e:
        logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None

## Treinar Spacy

In [0]:
def train_spacy():
    TRAIN_DATA = convert_dataturks_to_spacy('/content/drive/My Drive/Academico/8º Período/Mineração de Texto/Projeto de MT/Database/Rotulados/mineracao_mulheres.json')
    print(TRAIN_DATA[0])
    nlp = spacy.blank('pt')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(1):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

## Teste do modelo

In [0]:
my_model = train_spacy()

with open('/content/drive/My Drive/Academico/8º Período/Mineração de Texto/Projeto de MT/Database/Dados para teste/andamentos') as json_file:
    data = json.load(json_file)

i = 0
for text in data:
    doc = my_model(text['texto'])
    displacy.render(doc, style='ent',jupyter=True)
    
    i += 1
    if i == 20: break

('"texto": "Determinação de citação e intimação de partes e advogados 3ª Vara de Violência Doméstica e Familiar contra a Mulher da Capital\\r\\n\\r\\n         Processo nº.  0001140-91.2018.8.17.0001\\r\\n         Dia do fato: 16/12/2017\\r\\n\\r\\n         Despacho\\r\\n\\r\\n     Tendo em vista a manifestação do MP de fls. 18, determino a intimação da requerente para, querendo, ajuizar a presente queixa-crime no prazo legal.\\r\\n     \\r\\n     Decorrido o prazo sem resposta, certifique-se, voltando-me conclusos.\\r\\n     \\r\\n     Cumpra-se.\\r\\n     \\r\\n     Demais providencias legais.\\r\\n     \\r\\n     Recife, 07/02/2018\\r\\n\\r\\n\\r\\n          Maria Eliane Cabral Campos Carvalho\\r\\n                           Juíza de Direito"', {'entities': [(697, 702, 'juiz'), (631, 666, 'PER'), (599, 609, 'data'), (218, 228, 'data'), (167, 192, 'numero_processo'), (68, 125, 'vara')]})
Statring iteration 0
{'ner': 8837.711470912116}


In [0]:
#TRAIN_DATA = convert_dataturks_to_spacy('/content/drive/My Drive/Academico/8º Período/Mineração de Texto/Projeto de MT/Database/Rotulados/mineracao_mulheres.json')
#NEW_LABELS = ['advogado', 'vitima', 'juiz', 'sentenca', 'vara', 'numero_processo', 'crime_cometido', 'data',\
#             'numero_mpu', 'nome', 'interrupcao_processo','agressor', 'motivo_sentenca',\
#             'decreto', 'oab']

#print(TRAIN_DATA[0])

my_model = spacy.load('pt')
# my_model = train_spacy()

with open('/content/drive/My Drive/Academico/8º Período/Mineração de Texto/Projeto de MT/Database/Dados para teste/andamentos') as json_file:
    data = json.load(json_file)

i = 0
for text in data:
    doc = my_model(text['texto'])
    displacy.render(doc, style='ent',jupyter=True)
    
    i += 1
    if i == 10:
        break